In [30]:
import pandas as pd
import numpy as np
import string
import re

import nltk
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud
import matplotlib.pyplot as plt

import spacy
from spacy.training import Example
import random

from datasets import Dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

import sklearn
import numpy as np
import torch


# Ensure you have the necessary NLTK tokenizer models downloaded
# nltk.download('punkt')

In [31]:
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import TFAutoModelForTokenClassification
from transformers import create_optimizer
import tensorflow as tf
from transformers import DataCollatorForTokenClassification
from transformers.keras_callbacks import KerasMetricCallback

In [32]:
# from google.colab import drive
# drive.mount('/content/drive')

# folder_path = '/content/drive/MyDrive/UU/Thesis'

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
# Load an Excel file into a DataFrame
df = pd.read_excel('LCReviewsIntegrated_1962-1994.xlsx', engine='openpyxl')
# df = pd.read_excel(folder_path + '/LCReviewsIntegrated_1962-1994.xlsx', engine='openpyxl')

In [35]:
def remove_extra_spaces(text):
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

In [36]:
df['content'] = df['content'].apply(remove_extra_spaces)
df['title1'] = df['title1'].apply(remove_extra_spaces)

In [37]:
def remove_punctuation(input_string):
    # Create a translation table that maps each punctuation character to None
    translator = str.maketrans('', '', string.punctuation)
    # Translate the input string using the translation table
    return input_string.translate(translator)

In [38]:
def extract_title(x):
    if not (type(x.title1) == str):
        return np.nan
    if not (type(x.content) == str):
        return np.nan
    if x.title1.lower() in x.content.lower():
        return x.title1
    
    sentence_parts = re.split(r' / | : ', x.title1.lower())
    sentence_parts = sorted(sentence_parts, key=len, reverse=True)
    for part in sentence_parts:
        if part in x.content.lower():
            return part
        elif remove_punctuation(part) in x.content.lower():
            return remove_punctuation(part)

    if remove_punctuation(x.title1).lower() in x.content.lower():
        return remove_punctuation(x.title1)
        
    return "error"

In [39]:
def find_sentence_in_text(full_text, sentence):
    start_index = full_text.find(sentence)
    if start_index == -1:
        print("EROR!!!")
        return False
    end_index = start_index + len(sentence)
    return start_index, end_index

In [40]:
result = []
for index, row in df.iterrows():
    result.append(extract_title(row))

df['title3'] = result

In [41]:
df_clean = df[df['title3'] != 'error']

## Check what tokens are present before the title

In [42]:
def create_mask_for_sentence(full_text, sentence):
    # Tokenize the full text
    tokens = word_tokenize(full_text)
    # Find the start and end indices of the sentence in the full text
    start_index = full_text.find(sentence)
    if start_index == -1:
        return None, None
    end_index = start_index + len(sentence)
    # Tokenize the sentence separately to match tokens exactly
    sentence_tokens = word_tokenize(sentence)
    # Initialize mask with zeros
    mask = [0] * len(tokens)
    # Loop through the full text tokens to set the mask
    sentence_pos = 0
    for i, token in enumerate(tokens):
        if sentence_pos < len(sentence_tokens) and token == sentence_tokens[sentence_pos]:
            # Check if the full sequence matches and ensure it doesn't go out of bounds
            if i + len(sentence_tokens) - sentence_pos <= len(tokens) and \
                all(tokens[i + j] == sentence_tokens[sentence_pos + j] for j in range(len(sentence_tokens) - sentence_pos)):
                # Mark the mask for the length of the sentence tokens
                mask[i:i + len(sentence_tokens)] = [1] * len(sentence_tokens)
                break
            else:
                # Increment if it's not a full sequence match
                sentence_pos += 1
        elif token == sentence_tokens[0]:  # Reset if it's the beginning of sentence tokens
            sentence_pos = 0
    return tokens, mask

In [43]:
task = "ner"  # Should be one of "ner", "pos" or "chunk"
# model_checkpoint = "distilbert-base-uncased"
model_checkpoint = "Babelscape/wikineural-multilingual-ner"
batch_size = 16

In [44]:
label_list = ['O', 'I']

In [45]:
def create_data_set(samples):
    data = []

    for sample in samples:
        unique_content_df = df_clean[df_clean['content'] == sample]
        masks = []
    
        review = remove_punctuation(sample.lower())
    
        for index, row in unique_content_df.iterrows():
            book = remove_punctuation(row['title3'].lower())
            
            tokens, mask = create_mask_for_sentence(review, book)  # Assuming this returns a mask of the same length as tokens
            masks.append(mask)
        masks = np.bitwise_or.reduce(np.array(masks), axis=0)
        mask[mask == 0] = "O"
        mask[mask == 1] = "I"
        
        data.append({
            "tokens": tokens,
            "ner_tags": masks
        })

    return data

In [55]:
samples = np.random.choice(df_clean['content'].unique(), size=50, replace=False)
# samples = df_clean['content'].unique()[:10000]

train_dataset = Dataset.from_list(create_data_set(samples[:int(len(samples) * 0.8)]))
val_dataset = Dataset.from_list(create_data_set(samples[int(len(samples) * 0.8):]))

In [56]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [57]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [58]:
tokenized_dataset_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_dataset_val = val_dataset.map(tokenize_and_align_labels, batched=True)

tokenized_dataset_train

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 129.33 examples/s]


Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 40
})

In [59]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)
    ]
    
    total_count_O = 0
    total_count_I = 0
    for pred in true_predictions:
        total_count_O = total_count_O + sum(s.count("O") for s in pred)
        total_count_I = total_count_I + sum(s.count("I") for s in pred)



    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "total_count_O": total_count_O,
        "total_count_I": total_count_I,
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

C:\Users\Niels\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [60]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [61]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=2, ignore_mismatched_sizes=True
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
import torch.nn as nn

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, logits, targets):
        # Flatten predictions and labels
        logits = logits.view(-1, 2)  # Assuming 2 classes
        targets = targets.view(-1)

        # Masking -100 labels
        valid_indices = (targets != -100)
        logits = logits[valid_indices]
        targets = targets[valid_indices]

        intersection = torch.sum(logits[:, 1] * targets)
        union = torch.sum(logits[:, 1]) + torch.sum(targets)

        dice_score = (2. * intersection + self.smooth) / (union + self.smooth)
        dice_loss = 1 - dice_score

        return dice_loss

In [79]:
class MyTrainer(Trainer):
    # def compute_loss(self, model, inputs, return_outputs=False):
    # """ Custom weighted CrossEntropyLoss"""
    #     labels = inputs.pop("labels")
    #     outputs = model(**inputs)
    #     logits = outputs[0]

    #     # Reshape logits to [batch_size * sequence_length, num_classes]
    #     logits = logits.view(-1, logits.size(-1))
        
    #     # Reshape labels to [batch_size * sequence_length]
    #     labels = labels.view(-1)
        
    #     class_weights = torch.tensor([0.5, 28.27], dtype=torch.float32)
    #     loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
        
    #     loss = loss_fn(logits, labels)
        
    #     return (loss, outputs) if return_outputs else loss

    def compute_loss(self, model, inputs, return_outputs=False):
        """ DICE LOSS """
        # TODO: IGNORE -100 labels
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits  # Assuming logits are stored in outputs.logits
    
        # Compute loss using DiceLoss
        loss_fn = DiceLoss(smooth=1.2)
        loss = loss_fn(logits, labels)
    
        return (loss, outputs) if return_outputs else loss

In [88]:
training_args = TrainingArguments(
    output_dir="test_model",
    learning_rate=2e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    push_to_hub=False
)

In [89]:
# Create a custom Trainer instance
trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\Niels\miniconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [90]:
# del df
# del df_clean
# del train_dataset
# del val_dataset

In [91]:
trainer.train()

Epoch,Training Loss,Validation Loss,Total Count O,Total Count I,F1,Accuracy
1,No log,0.868758,0,4947,0.000000,0.010714
2,No log,0.979026,0,4947,0.000000,0.010714


TrainOutput(global_step=8, training_loss=0.9679167866706848, metrics={'train_runtime': 274.2062, 'train_samples_per_second': 0.292, 'train_steps_per_second': 0.029, 'total_flos': 20903740538880.0, 'train_loss': 0.9679167866706848, 'epoch': 2.0})